In [ ]:
!pip install mne 

In [ ]:
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew , variation
from scipy.signal import welch
import pandas as pd
from scipy.signal import hilbert
from collections import Counter

In [ ]:
%matplotlib qt

In [ ]:
plt.style.use('fast')

In [ ]:
import os

In [ ]:
os.chdir('C:/Users/dasha/Documents/Neurobiology_EEG/VFT/')

In [ ]:
import glob

In [ ]:
ID_for=['NP022309',
'NP042307',
'NP042312',
'NP042313',
'NP042315',
'NP042316',
'NP042317',
'NP042318',
'NP042319',
'NP042320',
'NP042321',
'NP042322',
'NP042324',
'NP042326',
'NP042331',
'NP052304',
'NP052305',
'NP052306',
'NP052307',
'NP052308',
'NP052312',
'NP052313',
'NP052314',
'NP052315',
'NP062304',
'NP062305']

In [ ]:
ID_inv=['NP022309',
'NP042307',
'NP042310',
'NP042312',
'NP042313',
'NP042315',
'NP042316',
'NP042318',
'NP042320',
'NP042321',
'NP042322',
'NP042323',
'NP042326',
'NP042331',
'NP052304',
'NP052307',
'NP052308',
'NP052309',
'NP052311',
'NP052312',
'NP052313',
'NP052314',
'NP052315',
'NP062301',
'NP062304']

In [ ]:
ID_ok=['NP042307',
'NP042312',
'NP042313',
'NP042315',
'NP042316',
'NP042318',
'NP042320',
'NP042321',
'NP042322',
'NP042326',
'NP042331',
'NP052304',
'NP052307',
'NP052308',
'NP052312',
'NP052313',
'NP052314',
'NP052315',
'NP062304',
'NP102303',
'NP102304',
'NP102305',
'NP102306',
'NP102310',
'NP102311',
'NP102312',
'NP102314',
'NP112302',
'NP112303',
'NP112304',
'NP112306',
'NP112308',
'NP112310',
'NP112313',
'NP112315',
'NP112316',
'NP112317',
'NP122301',
'NP122303']

In [ ]:
filenames = glob.glob('raw/*_VFT_2.Fif')
#filenames = glob.glob('*.')
filenames

In [ ]:
len(filenames)

In [ ]:
ID=[]
for f in filenames:
    ID.append(f[4:-10])
ID

In [ ]:
EEG = []

for i in range (len(ID)):  
    #name= f'{ID[i]}_VFT.vhdr'
    name=filenames[i]
    E = mne.io.read_raw_fif(name, verbose=False, preload=True)
    EEG.append(E)

In [ ]:
for i in range (0, len(EEG)):
    EEG[i]=EEG[i].drop_channels(['Photo', 'micr'])    
    EEG[i]=EEG[i].filter(0.1, 100, phase='zero-double')
    EEG[i] =  EEG[i].notch_filter(50, verbose=None)
    EEG[i].plot(start=0, n_channels=len(EEG[i].ch_names))

In [ ]:
for i in range (len(EEG)):
    print(ID[i])
    print(EEG[i].info['bads'])

In [ ]:
for i in range (len(ID)):
  EEG[i]= EEG[i].interpolate_bads(reset_bads=True, method=dict(eeg='MNE'), verbose=True)
    

In [ ]:
for i in range (len(EEG)):
    EEG[i]=EEG[i].set_eeg_reference(ref_channels='average')

In [ ]:
Evlist = []
for i in range (0, len(EEG)):
    #print(ID_short[i])
    events=mne.events_from_annotations(EEG[i]) 
    Evlist.append(events)

In [ ]:
event_dict = {'Start': 1, 'End': 11}
mne.viz.plot_events(Evlist[4][0], event_id=event_dict)

In [ ]:
directory = "procested"
os.mkdir(directory) 

In [ ]:
for i in range (len(ID)):
  EEG[i].save('procested/'+ID[i] + '.fif', overwrite=True)

In [ ]:
crop_list=[]
for i in range (0, len(EEG)):
    start=Evlist[i][0][1][0]/500
    print(start)
    ep = EEG[i].copy().crop(tmin=start, tmax=start+120)
    crop_list.append(ep)

In [ ]:
for i in range (len(ID)):
    crop_list[i].save('procested/cut/for/'+ID[i] + '.fif', overwrite=True)

In [ ]:
crop_list=[]
for i in range (0, len(EEG)):
    start=Evlist[i][0][3][0]/500
    print(start)
    ep = EEG[i].copy().crop(tmin=start, tmax=start+120)
    crop_list.append(ep)

In [ ]:
for i in range (len(ID)):
    crop_list[i].save('procested/cut/inv/'+ID[i] + '.fif', overwrite=True)

In [ ]:
def z_spectrum(psd, noise_n_neighborfreqs=1, noise_skip_neighborfreqs=1):
    """
    Parameters
    ----------
    psd - np.array
        containing psd values as spit out by mne functions. must be 2d or 3d
        with frequencies in the last dimension
    noise_n_neighborfreqs - int
        number of neighboring frequencies used to compute noise level.
        increment by one to add one frequency bin ON BOTH SIDES
    noise_skip_neighborfreqs - int
        set this >=1 if you want to exclude the immediately neighboring
        frequency bins in noise level calculation

    Returns
    -------
    snr - np.array
        array containing snr for all epochs, channels, frequency bins.
        NaN for frequencies on the edge, that do not have enoug neighbors on
        one side to calculate snr
    """

    # prep not epoched / single channel data
    is_2d = True if (len(psd.shape) == 2) else False
    if is_2d:
        psd = psd.reshape((1, psd.shape[0], psd.shape[1]))

    # SNR loop
    z_scr = np.empty(psd.shape)
    for i_freq in range(psd.shape[2]):

        # skip freqs on the edges (without noise neighbors)

        start_freq_i = noise_n_neighborfreqs + noise_skip_neighborfreqs
        stop_freq_i = (psd.shape[2] - noise_n_neighborfreqs
                       - noise_skip_neighborfreqs)
        if not (stop_freq_i > i_freq >= start_freq_i):
            z_scr[:, :, i_freq] = np.nan
            continue

        # extract signal level
        signal = psd[:, :, i_freq]

        # ... and average noise level
        i_noise = []
        for i in range(noise_n_neighborfreqs):
            i_noise.append(i_freq + noise_skip_neighborfreqs + i + 1)
            i_noise.append(i_freq - noise_skip_neighborfreqs - i - 1)
        noise_mean = psd[:, :, i_noise].mean(axis=2)
        noise_std = psd[:, :, i_noise].std(axis=2)

        z_scr[:, :, i_freq] = (signal - noise_mean) / noise_std

    # reshape not epoched / single channel data to original dimensions
    if is_2d:
        z_scr = z_scr.reshape(z_scr.shape[1], z_scr.shape[2])

    return z_scr

In [ ]:
def snr_spectrum(psd, noise_n_neighborfreqs=1, noise_skip_neighborfreqs=1):
    """
    Parameters
    ----------
    psd - np.array
        containing psd values as spit out by mne functions. must be 2d or 3d
        with frequencies in the last dimension
    noise_n_neighborfreqs - int
        number of neighboring frequencies used to compute noise level.
        increment by one to add one frequency bin ON BOTH SIDES
    noise_skip_neighborfreqs - int
        set this >=1 if you want to exclude the immediately neighboring
        frequency bins in noise level calculation

    Returns
    -------
    snr - np.array
        array containing snr for all epochs, channels, frequency bins.
        NaN for frequencies on the edge, that do not have enoug neighbors on
        one side to calculate snr
    """

    # prep not epoched / single channel data
    is_2d = True if (len(psd.shape) == 2) else False
    if is_2d:
        psd = psd.reshape((1, psd.shape[0], psd.shape[1]))

    # SNR loop
    snr = np.empty(psd.shape)
    for i_freq in range(psd.shape[2]):

        # skip freqs on the edges (without noise neighbors)

        start_freq_i = noise_n_neighborfreqs + noise_skip_neighborfreqs
        stop_freq_i = (psd.shape[2] - noise_n_neighborfreqs
                       - noise_skip_neighborfreqs)
        if not (stop_freq_i > i_freq >= start_freq_i):
            snr[:, :, i_freq] = np.nan
            continue

        # extract signal level
        signal = psd[:, :, i_freq]

        # ... and average noise level
        i_noise = []
        for i in range(noise_n_neighborfreqs):
            i_noise.append(i_freq + noise_skip_neighborfreqs + i + 1)
            i_noise.append(i_freq - noise_skip_neighborfreqs - i - 1)
        noise = psd[:, :, i_noise].mean(axis=2)

        snr[:, :, i_freq] = signal / noise

    # reshape not epoched / single channel data to original dimensions
    if is_2d:
        snr = snr.reshape(snr.shape[1], snr.shape[2])

    return snr

In [ ]:
def bas_cor(psd, noise_n_neighborfreqs=1, noise_skip_neighborfreqs=1):
    """
    Parameters
    ----------
    psd - np.array
        containing psd values as spit out by mne functions. must be 2d or 3d
        with frequencies in the last dimension
    noise_n_neighborfreqs - int
        number of neighboring frequencies used to compute noise level.
        increment by one to add one frequency bin ON BOTH SIDES
    noise_skip_neighborfreqs - int
        set this >=1 if you want to exclude the immediately neighboring
        frequency bins in noise level calculation

    Returns
    -------
    snr - np.array
        array containing snr for all epochs, channels, frequency bins.
        NaN for frequencies on the edge, that do not have enoug neighbors on
        one side to calculate snr
    """

    # prep not epoched / single channel data
    is_2d = True if (len(psd.shape) == 2) else False
    if is_2d:
        psd = psd.reshape((1, psd.shape[0], psd.shape[1]))

    # SNR loop
    bas_cor_spect = np.empty(psd.shape)
    for i_freq in range(psd.shape[2]):

        # skip freqs on the edges (without noise neighbors)

        start_freq_i = noise_n_neighborfreqs + noise_skip_neighborfreqs
        stop_freq_i = (psd.shape[2] - noise_n_neighborfreqs
                       - noise_skip_neighborfreqs)
        if not (stop_freq_i > i_freq >= start_freq_i):
            bas_cor_spect[:, :, i_freq] = np.nan
            continue

        # extract signal level
        signal = psd[:, :, i_freq]

        # ... and average noise level
        i_noise = []
        for i in range(noise_n_neighborfreqs):
            i_noise.append(i_freq + noise_skip_neighborfreqs + i + 1)
            i_noise.append(i_freq - noise_skip_neighborfreqs - i - 1)
        noise_mean = psd[:, :, i_noise].mean(axis=2)
        noise_std = psd[:, :, i_noise].std(axis=2)

        bas_cor_spect[:, :, i_freq] = signal - noise_mean

    # reshape not epoched / single channel data to original dimensions
    if is_2d:
        bas_cor_spect = bas_cor_spect.reshape(bas_cor_spect.shape[1], bas_cor_spect.shape[2])

    return bas_cor_spect

In [ ]:
#target_list=eplist
tmin = 0
tmax = 60.
fmin = 0.1
fmax = 100.
sfreq = 500
psds_list = []
snrs_list = []
zscrs_list = []
amp_cor_list = []

for i in range (0, len(EEG)):
    data=H1[i].get_data()
    fs = 500
    fft_vals = np.absolute(np.fft.rfft(data))
    fft_freq = np.fft.rfftfreq(len(data[0]), 1.0/fs)

    psds_list.append(fft_vals)
    freqs_list.append(fft_freq)

    snrs = snr_spectrum(fft_vals, 10, 1)
    snrs_list.append(snrs)  
    zscrs = z_spectrum(fft_vals, 10, 1)
    zscrs_list.append(zscrs)
    bc = bas_cor(fft_vals, 10, 1)
    amp_cor_list.append(bc)           
    
av_psds = np.mean(np.array(psds_list), axis=0)
av_freqs = np.mean(np.array(freqs_list), axis=0)
av_snrs = np.mean(snrs_list, axis=0)   
av_zscrs = np.mean(zscrs_list, axis=0) 
av_cors = np.mean(amp_cor_list, axis=0) 

fig, axes = plt.subplots(2, 1, sharex='all', sharey='none', figsize=(18, 10))
freq_range = range(np.where(np.floor(av_freqs) == 1.)[0][0],
                   np.where(np.ceil(av_freqs) == fmax - 1)[0][0])

psds_plot = av_psds
psds_mean = psds_plot.mean(axis=(0))[freq_range]
psds_std = psds_plot.std(axis=(0))[freq_range]
axes[0].plot(av_freqs[freq_range], psds_mean, color='C2', linewidth=3)
axes[0].fill_between(
    av_freqs[freq_range], psds_mean - psds_std, psds_mean + psds_std,
    color='C3', alpha=.2)
axes[0].set(title="Amplitude spectrum", ylabel='Amplitude [mV]')

# SNR spectrum
snr_mean = av_snrs.mean(axis=(0))[freq_range]
snr_std = av_snrs.std(axis=(0))[freq_range]

axes[1].plot(av_freqs[freq_range], snr_mean, color='C5',linewidth=3)
axes[1].fill_between(
    av_freqs[freq_range], snr_mean - snr_std, snr_mean + snr_std,
    color='C5', alpha=.2)
axes[1].set(
    title="SNR spectrum", xlabel='Frequency [Hz]',
    ylabel='SNR', ylim=[0.8, 3], xlim=[fmin, 14])

axes[0].axvspan(1.2-0.03, 1.2+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*2-0.03, 1.2*2+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*3-0.03, 1.2*3+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*4-0.03, 1.2*4+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(6-0.03, 6+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*2-0.03, 6*2+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*3-0.03, 6*3+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*4-0.03, 6*4+0.03, alpha=0.2, color ='darkblue')

axes[1].axvspan(1.2-0.03, 1.2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*2-0.03, 1.2*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*3-0.03, 1.2*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*4-0.03, 1.2*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(6-0.03, 6+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*2-0.03, 6*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*3-0.03, 6*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*4-0.03, 6*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
fig.suptitle('H1', fontsize=16)

In [ ]:
fig, axes = plt.subplots(2, 1, sharex='all', sharey='none', figsize=(18, 10))
freq_range = range(np.where(np.floor(av_freqs) == 1.)[0][0],
                   np.where(np.ceil(av_freqs) == fmax - 1)[0][0])

psds_plot = av_psds
psds_mean = psds_plot.mean(axis=(0))[freq_range]
psds_std = psds_plot.std(axis=(0))[freq_range]
axes[0].plot(av_freqs[freq_range], psds_mean, color='C2', linewidth=3)
axes[0].fill_between(
    av_freqs[freq_range], psds_mean - psds_std, psds_mean + psds_std,
    color='C3', alpha=.2)
axes[0].set(title="Amplitude spectrum", ylabel='Amplitude [mV]')

# SNR spectrum
snr_mean = av_snrs.mean(axis=(0))[freq_range]
snr_std = av_snrs.std(axis=(0))[freq_range]

axes[1].plot(av_freqs[freq_range], snr_mean, color='C5',linewidth=3)
axes[1].fill_between(
    av_freqs[freq_range], snr_mean - snr_std, snr_mean + snr_std,
    color='C5', alpha=.2)
axes[1].set(
    title="SNR spectrum", xlabel='Frequency [Hz]',
    ylabel='SNR', ylim=[0.8, 3], xlim=[fmin, 14])

axes[0].axvspan(1.2-0.03, 1.2+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*2-0.03, 1.2*2+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*3-0.03, 1.2*3+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*4-0.03, 1.2*4+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(6-0.03, 6+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*2-0.03, 6*2+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*3-0.03, 6*3+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*4-0.03, 6*4+0.03, alpha=0.2, color ='darkblue')

axes[1].axvspan(1.2-0.03, 1.2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*2-0.03, 1.2*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*3-0.03, 1.2*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*4-0.03, 1.2*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(6-0.03, 6+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*2-0.03, 6*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*3-0.03, 6*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*4-0.03, 6*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')


In [ ]:
#target_list=eplist
tmin = 0
tmax = 120.
fmin = 0.1
fmax = 100.
sfreq = 500
psds_list = []
freqs_list = []
snrs_list = []
zscrs_list = []
amp_cor_list = []

for i in range (0, len(EEG)):
    data=EEG[i].get_data(units='uV')
    fs = 500
    fft_vals = np.absolute(np.fft.rfft(data))/(len(data[0])/2)
    fft_freq = np.fft.rfftfreq(len(data[0]), 1.0/fs)

    psds_list.append(fft_vals)
    freqs_list.append(fft_freq)

    snrs = snr_spectrum(fft_vals, 10, 1)
    snrs_list.append(snrs)  
    zscrs = z_spectrum(fft_vals, 10, 1)
    zscrs_list.append(zscrs)
    bc = bas_cor(fft_vals, 10, 1)
    amp_cor_list.append(bc) 
    print(f'{i+1} from {len(EEG)} done')
    
av_psds = np.mean(psds_list, axis=0)
av_freqs = np.mean(freqs_list, axis=0)
av_snrs = np.mean(snrs_list, axis=0)   
av_zscrs = np.mean(zscrs_list, axis=0) 
av_cors = np.mean(amp_cor_list, axis=0) 

In [ ]:
from numpy import save
from numpy import load

In [ ]:
np.concatenate(psds_list, axis=0)
save('for_psds.npy', psds_list)
print('done')
np.concatenate(freqs_list, axis=0)
save('for_freqs.npy', freqs_list)
print('done')
np.concatenate(snrs_list, axis=0)
save('for_snrs.npy', snrs_list)
print('done')
np.concatenate(zscrs_list, axis=0)
save('for_zscrs.npy', zscrs_list)
print('done')
np.concatenate(amp_cor_list, axis=0)
save('for_amp_cor.npy', amp_cor_list)
print('done')

In [ ]:
print('done')

In [ ]:
from scipy.fft import fft, fftfreq

In [ ]:
from scipy.io import loadmat
from scipy import signal

In [ ]:
fig, axes = plt.subplots(2, 1, sharex='all', sharey='none', figsize=(18, 10))
freq_range = range(np.where(np.floor(fft_freq) == 1.)[0][0],
                   np.where(np.ceil(fft_freq) == fmax - 1)[0][0])

psds_mean = fft_vals.mean(axis=(0))[freq_range]
psds_std = fft_vals.std(axis=(0))[freq_range]
axes[0].plot(fft_freq[freq_range], psds_mean, color='C2')
axes[0].fill_between(
    fft_freq[freq_range], psds_mean - psds_std, psds_mean + psds_std,
    color='C2', alpha=.2)
axes[0].set(title="PSD spectrum", ylabel='Power Spectral Density [dB]')

# SNR spectrum
snr_mean = snrs.mean(axis=(0))[freq_range]
snr_std = snrs.std(axis=(0))[freq_range]

axes[1].plot(fft_freq[freq_range], snr_mean, color='C0')
axes[1].fill_between(
    fft_freq[freq_range], snr_mean - snr_std, snr_mean + snr_std,
    color='C0', alpha=.2)
axes[1].set(
    title="SNR spectrum", xlabel='Frequency [Hz]',
    ylabel='SNR', ylim=[0, 10], xlim=[fmin, 15])
axes[0].axvspan(1.2-0.03, 1.2+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*2-0.03, 1.2*2+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*3-0.03, 1.2*3+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(1.2*4-0.03, 1.2*4+0.03, alpha=0.2, color ='orchid')
axes[0].axvspan(6-0.03, 6+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*2-0.03, 6*2+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*3-0.03, 6*3+0.03, alpha=0.2, color ='darkblue')
axes[0].axvspan(6*4-0.03, 6*4+0.03, alpha=0.2, color ='darkblue')

axes[1].axvspan(1.2-0.03, 1.2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*2-0.03, 1.2*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*3-0.03, 1.2*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(1.2*4-0.03, 1.2*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
axes[1].axvspan(6-0.03, 6+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*2-0.03, 6*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*3-0.03, 6*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
axes[1].axvspan(6*4-0.03, 6*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')

In [ ]:
for i in range (len(ID)):
    fig, axes = plt.subplots(2, 1, sharex='all', sharey='none', figsize=(18, 10))
    freq_range = range(np.where(np.floor(av_freqs) == 1.)[0][0],
                   np.where(np.ceil(av_freqs) == fmax - 1)[0][0])

    psds_plot = psds_list[i]
    psds_mean = psds_plot.mean(axis=(0))[freq_range]
    psds_std = psds_plot.std(axis=(0))[freq_range]
    axes[0].plot(av_freqs[freq_range], psds_mean, color='C3', linewidth=3)
    axes[0].fill_between(
        av_freqs[freq_range], psds_mean - psds_std, psds_mean + psds_std,
        color='C3', alpha=.2)
    axes[0].set(title="Amplitude spectrum", ylabel='Amplitude [mV]')

# SNR spectrum
    snr_mean = snrs_list[i].mean(axis=(0))[freq_range]
    snr_std = snrs_list[i].std(axis=(0))[freq_range]

    axes[1].plot(av_freqs[freq_range], snr_mean, color='C4',linewidth=3)
    axes[1].fill_between(
        av_freqs[freq_range], snr_mean - snr_std, snr_mean + snr_std,
        color='C5', alpha=.2)
    axes[1].set(
        title="SNR spectrum", xlabel='Frequency [Hz]',
        ylabel='SNR', ylim=[0.8, 5], xlim=[fmin, 25])

    axes[0].axvspan(1.2-0.03, 1.2+0.03, alpha=0.2, color ='orchid')
    axes[0].axvspan(1.2*2-0.03, 1.2*2+0.03, alpha=0.2, color ='orchid')
    axes[0].axvspan(1.2*3-0.03, 1.2*3+0.03, alpha=0.2, color ='orchid')
    axes[0].axvspan(1.2*4-0.03, 1.2*4+0.03, alpha=0.2, color ='orchid')
    axes[0].axvspan(6-0.03, 6+0.03, alpha=0.2, color ='darkblue')
    axes[0].axvspan(6*2-0.03, 6*2+0.03, alpha=0.2, color ='darkblue')
    axes[0].axvspan(6*3-0.03, 6*3+0.03, alpha=0.2, color ='darkblue')
    axes[0].axvspan(6*4-0.03, 6*4+0.03, alpha=0.2, color ='darkblue')

    axes[1].axvspan(1.2-0.03, 1.2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
    axes[1].axvspan(1.2*2-0.03, 1.2*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
    axes[1].axvspan(1.2*3-0.03, 1.2*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
    axes[1].axvspan(1.2*4-0.03, 1.2*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='orchid')
    axes[1].axvspan(6-0.03, 6+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
    axes[1].axvspan(6*2-0.03, 6*2+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
    axes[1].axvspan(6*3-0.03, 6*3+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
    axes[1].axvspan(6*4-0.03, 6*4+0.03, ymin=0, ymax=5, alpha=0.2, color ='darkblue')
    fig.suptitle(f'{ID[i]}', fontsize=16)#В ковычках можем прописать заголовок
    fig.show()
    fig.savefig(f'{ID[i]}_for_spectr.png')

In [ ]:
fft_vals.mean(axis=(0))[freq_range]

In [ ]:
Nyquist = 50/2
print('Nyquist frequency = %2.4f Hz'%Nyquist)
Nsamples = int( math.floor(eeg.size/2) )
hz = np.linspace(0, Nyquist, num = Nsamples + 1 )
dhz = hz[1]
print('Spectral resolution = %2.4f Hz'%hz[1])

In [ ]:
frq=[1.2,2.4,3.6,4.8,6.0,7.2]
for f in frq:
    ch_list=[]
    frq_ind=np.argmin(abs(av_freqs - f))
    t_snrs = av_snrs[:, :, frq_ind].mean(axis=0)
    mask=np.asarray(t_snrs>1.6).nonzero()
    for a in mask[0]:
        ch_list.append(EEG[0].ch_names[a])
    print(f'{f}Hz ({int(f/1.2)}f/5) sig chan:{ch_list}')

In [ ]:
frq=[6,12,18,24,30,36]
for f in frq:
    ch_list=[]
    frq_ind=np.argmin(abs(av_freqs - f))
    t_snrs = av_snrs[:, :, frq_ind].mean(axis=0)
    mask=np.asarray(t_snrs>1.64).nonzero()
    for a in mask[0]:
        ch_list.append(EEG[0].ch_names[a])
    print(f'{f}Hz ({int(f/6)}f) sig chan:{ch_list}')

In [ ]:
av_for_zscrs = np.mean(for_zscrs_list, axis=0)   
av_inv_zscrs = np.mean(inv_zscrs_list, axis=0)   
frq=[1.2,2.4,3.6,4.8,6.0,7.2,8.4,9.6,10.8]
for f in frq:
    frq_ind=np.argmin(abs(av_freqs - f))
    t_snrs = av_inv_zscrs[ :, frq_ind]
    mask=np.copy(t_snrs)
    n_s=len(np.asarray(t_snrs>1.64).nonzero()[0])
    np.place(mask, mask<1.64, [0])
    ind = (-mask).argsort()[:n_s]
    ch_sort=[EEG[0].ch_names[i] for i in ind]
    print(f'{f}Hz ({int(f/1.2)}f/5) sig chan:{ch_sort}')

In [ ]:
frq=[6,12,18,24,30,36,42,48,54]
for f in frq:
    frq_ind=np.argmin(abs(av_freqs - f))
    t_snrs = av_inv_zscrs[ :, frq_ind]
    mask=np.copy(t_snrs)
    n_s=len(np.asarray(t_snrs>1.64).nonzero()[0])
    np.place(mask, mask<1.64, [0])
    ind = (-mask).argsort()[:n_s]
    ch_sort=[EEG[0].ch_names[i] for i in ind]
    print(f'{f}Hz ({int(f/6)}f) sig chan:{ch_sort}')

In [ ]:
ch_P8=['P8']

ch_P7=['P7']

ch_O1=['O1']

ch_O2=['O2']
# Find corresponding indices using mne.pick_types()
picks_P8 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P8)
picks_P7 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P7)
picks_O1 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O1)
picks_O2 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O2)

zscrs_roi_P8 = av_zscrs[:, picks_P8, :].mean(axis=1)
zscrs_roi_P7 = av_zscrs[:, picks_P7, :].mean(axis=1)
zscrs_roi_O1 = av_zscrs[:, picks_O1, :].mean(axis=1)
zscrs_roi_O2 = av_zscrs[:, picks_O2, :].mean(axis=1)

freq_plot = [1.2, 2.4, 3.6, 4.8, 6, 12, 18, 24]
#color_plot = ['C5', 'C6']
xpos_plot = [ -0.8,-0.6,-0.4,-0.2, 0.2,0.4,0.6,0.8]
fig, ax = plt.subplots(figsize=[20, 10])
labels = ['P8', 'P7', 'O1', 'O2' ]
x = np.arange(len(labels))  # the label locations
width = 0.3 # the width of the bars
res = dict()

# loop to plot SNRs at stimulation frequencies and harmonics
for i, f in enumerate(freq_plot):
    # extract snrs
    stim_P8_tmp = \
        zscrs_roi_P8[:, np.argmin(abs(freqs - f))]
    stim_P7_tmp = \
        zscrs_roi_P7[:, np.argmin(abs(freqs - f))]
    stim_O1_tmp = \
        zscrs_roi_O1[:, np.argmin(abs(freqs - f))]
    stim_O2_tmp = \
        zscrs_roi_O2[:, np.argmin(abs(freqs - f))]
    
    SNR_tmp = [stim_P8_tmp.mean(), stim_P7_tmp.mean(), stim_O1_tmp.mean(), stim_O2_tmp.mean()]
    # plot (with std)
    ax.bar(
        x + width * xpos_plot[i], SNR_tmp, width / len(freq_plot),
        yerr=np.std(SNR_tmp),
        label='%i Hz Amp' % f)
    # store results for statistical comparison
    res['P8_amp_%ihz' % f] = stim_P8_tmp
    res['P7_amp_%ihz' % f] = stim_P7_tmp
    res['O1_amp_%ihz' % f] = stim_O1_tmp
    res['O2_amp_%ihz' % f] = stim_O2_tmp


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Amplitude')
ax.set_title(f'Average amplitude \n Foreign')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend([f'{f} Hz' for f in freq_plot], title='Amplitude at:')
ax.set_ylim([0, 30])
ax.axhline(1.64, ls='--', c='r')
fig.show()
#fig.savefig(f'Amp_Invers.png')

In [ ]:
ch_P8=['P8']

ch_P7=['P7']

ch_O1=['O1']

ch_O2=['O2']
# Find corresponding indices using mne.pick_types()
picks_P8 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P8)
picks_P7 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P7)
picks_O1 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O1)
picks_O2 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O2)

zscrs_roi_P8 = av_zscrs[:, picks_P8, :].mean(axis=1)
zscrs_roi_P7 = av_zscrs[:, picks_P7, :].mean(axis=1)
zscrs_roi_O1 = av_zscrs[:, picks_O1, :].mean(axis=1)
zscrs_roi_O2 = av_zscrs[:, picks_O2, :].mean(axis=1)

freq_plot = [1.2, 2.4, 6, 12]
#color_plot = ['C5', 'C6']
xpos_plot = [ -0.5,-0.2, 0.2, 0.5,]
fig, ax = plt.subplots(1,2, figsize=[22, 10])
labels_l = ['P7', 'O1',]
labels_r = ['P8', 'O2',]
x = np.arange(2)  # the label locations
width = 0.6 # the width of the bars
res = dict()

# loop to plot SNRs at stimulation frequencies and harmonics
for i, f in enumerate(freq_plot):
    # extract snrs
    stim_P8_tmp = \
        zscrs_roi_P8[:, np.argmin(abs(av_freqs - f))]
    stim_P7_tmp = \
        zscrs_roi_P7[:, np.argmin(abs(av_freqs - f))]
    stim_O1_tmp = \
        zscrs_roi_O1[:, np.argmin(abs(av_freqs - f))]
    stim_O2_tmp = \
        zscrs_roi_O2[:, np.argmin(abs(av_freqs - f))]
    
    SNR_tmp_l = [stim_P7_tmp.mean(), stim_O1_tmp.mean()]
    SNR_tmp_r = [stim_P8_tmp.mean(), stim_O2_tmp.mean()]
    
# plot (with std)
    ax[0].bar(
        x + width * xpos_plot[i], SNR_tmp_l, width / len(freq_plot),
        yerr=np.std(SNR_tmp),
        label='%i Hz Amp' % f)
    ax[1].bar(
        x + width * xpos_plot[i], SNR_tmp_r, width / len(freq_plot),
        yerr=np.std(SNR_tmp),
        label='%i Hz Amp' % f)
    # store results for statistical comparison
    res['P8_amp_%ihz' % f] = stim_P8_tmp
    res['P7_amp_%ihz' % f] = stim_P7_tmp
    res['O1_amp_%ihz' % f] = stim_O1_tmp
    res['O2_amp_%ihz' % f] = stim_O2_tmp


# Add some text for labels, title and custom x-axis tick labels, etc.
ax[0].set_ylabel('Amplitude')
ax[1].set_ylabel('Amplitude')
ax[0].set_title('Left hem')
ax[1].set_title('Right hem')
ax[0].set_xticks(x)
ax[1].set_xticks(x)
ax[0].set_xticklabels(labels_l)
ax[1].set_xticklabels(labels_r)
plt.legend([f'{f} Hz' for f in freq_plot], title='Amplitude at:')
ax[0].set_ylim([0, 10])
ax[1].set_ylim([0, 10])
ax[0].axhline(1.64, ls='--', c='r')
ax[1].axhline(1.64, ls='--', c='r')
fig.suptitle('Average amplitude \n Foreign', fontsize=22)
fig.show()

In [ ]:
wal = dict()
freq_plot = [1.2, 2.4, 3.6, 4.8, 6, 12, 18, 24]
#color_plot = ['C5', 'C6']
xpos_plot = [ -0.8,-0.6,-0.4,-0.2, 0.2,0.4,0.6,0.8]
labels = ['P8', 'P7', 'O1', 'O2' ]
x = np.arange(len(labels))  # the label locations
width = 0.3 
for j in range (0, len(EEG)):
    fig, ax = plt.subplots(figsize=[20, 10])
    
    res = dict()
    picks_P8 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P8)
    picks_P7 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P7)
    picks_O1 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O1)
    picks_O2 = mne.pick_types(eplist[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O2)

    zscrs_roi_P8 = amp_cor_list[j][:, picks_P8, :].mean(axis=1)
    zscrs_roi_P7 = amp_cor_list[j][:, picks_P7, :].mean(axis=1)
    zscrs_roi_O1 = amp_cor_list[j][:, picks_O1, :].mean(axis=1)
    zscrs_roi_O2 = amp_cor_list[j][:, picks_O2, :].mean(axis=1)
    
    for i, f in enumerate(freq_plot):
    # extract snrs
        stim_P8_tmp = \
            zscrs_roi_P8[:, np.argmin(abs(av_freqs - f))][0]
        stim_P7_tmp = \
            zscrs_roi_P7[:, np.argmin(abs(av_freqs - f))][0]
        stim_O1_tmp = \
            zscrs_roi_O1[:, np.argmin(abs(av_freqs - f))][0]
        stim_O2_tmp = \
            zscrs_roi_O2[:, np.argmin(abs(av_freqs - f))][0]
    
        SNR_tmp = [stim_P8_tmp, stim_P7_tmp, stim_O1_tmp, stim_O2_tmp]
    # plot (with std)
        ax.bar(
            x + width * xpos_plot[i], SNR_tmp, width / len(freq_plot),
            yerr=np.std(SNR_tmp),
            label='%i Hz Amp' % f)
    # store results for statistical comparison
        res['P8_amp_%ihz' % f] = stim_P8_tmp
        res['P7_amp_%ihz' % f] = stim_P7_tmp
        res['O1_amp_%ihz' % f] = stim_O1_tmp
        res['O2_amp_%ihz' % f] = stim_O2_tmp
        wal[ID[j]]=res
# Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel('Z score')
        ax.set_title(f'{ID[j]} \n Invers')
        ax.set_xticks(x)
        ax.set_xticklabels(labels)
        ax.legend([f'{f} Hz' for f in freq_plot], title='SNR at:')
        ax.set_ylim([0, 12])
        ax.axhline(1, ls='--', c='r')
        fig.show()
       # fig.savefig(f'individual/{ID[j]}_amp_chan.png')  

In [ ]:
O2_list

In [ ]:
psds_list=load('for_psds.npy')
freqs_list=load('for_freqs.npy')
snrs_list=load('for_snrs.npy')
zscrs_list=load('for_zscrs.npy')
amp_cor_list=load('for_amp_cor.npy')

wal = dict()
freq_plot = [[1.2, 2.4, 3.6],[6, 12, 18]]
ch_P8=['P8']
ch_P7=['P7']
ch_O1=['O1']
ch_O2=['O2']

for j in range (0, len(EEG)):    
    res = dict()
    picks_P8 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P8)
    picks_P7 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P7)
    picks_O1 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O1)
    picks_O2 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O2)

    zscrs_roi_P8 = amp_cor_list[j][ picks_P8, :]
    zscrs_roi_P7 = amp_cor_list[j][ picks_P7, :]
    zscrs_roi_O1 = amp_cor_list[j][ picks_O1, :]
    zscrs_roi_O2 = amp_cor_list[j][ picks_O2, :]
    P8_list=[]
    P7_list=[]
    O1_list=[]
    O2_list=[]
    for i, f in enumerate(freq_plot[0]):
    # extract snrs
        stim_P8_tmp = \
            zscrs_roi_P8[:, np.argmin(abs(av_freqs - f))][0]
        P8_list.append(stim_P8_tmp)
        stim_P7_tmp = \
            zscrs_roi_P7[:, np.argmin(abs(av_freqs - f))][0]
        P7_list.append(stim_P7_tmp)
        stim_O1_tmp = \
            zscrs_roi_O1[:, np.argmin(abs(av_freqs - f))][0]
        O1_list.append(stim_O1_tmp)
        stim_O2_tmp = \
            zscrs_roi_O2[:, np.argmin(abs(av_freqs - f))][0]
        O2_list.append(stim_O2_tmp)

    res['P8_amp_ob'] = sum(P8_list)
    res['P7_amp_ob']  = sum(P7_list)
    res['O1_amp_ob'] = sum(O1_list)
    res['O2_amp_ob'] = sum(O2_list)
    P8_list=[]
    P7_list=[]
    O1_list=[]
    O2_list=[]
    for i, f in enumerate(freq_plot[1]):

    # extract snrs
        stim_P8_tmp = \
            zscrs_roi_P8[:, np.argmin(abs(av_freqs - f))][0]
        P8_list.append(stim_P8_tmp)
        stim_P7_tmp = \
            zscrs_roi_P7[:, np.argmin(abs(av_freqs - f))][0]
        P7_list.append(stim_P7_tmp)
        stim_O1_tmp = \
            zscrs_roi_O1[:, np.argmin(abs(av_freqs - f))][0]
        O1_list.append(stim_O1_tmp)
        stim_O2_tmp = \
            zscrs_roi_O2[:, np.argmin(abs(av_freqs - f))][0]
        O2_list.append(stim_O2_tmp)

    res['P8_amp_bas'] = sum(P8_list)
    res['P7_amp_bas']  = sum(P7_list)
    res['O1_amp_bas'] = sum(O1_list)
    res['O2_amp_bas'] = sum(O2_list)
    wal[ID[j]]=res

df=pd.DataFrame.from_dict(wal, orient='index')
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'ID'})
df = pd.melt(df, id_vars=['ID'], value_vars=['P8_amp_ob', 'P7_amp_ob', 'O1_amp_ob', 'O2_amp_ob', 
                                             'P8_amp_bas', 'P7_amp_bas', 'O1_amp_bas', 'O2_amp_bas'],
        var_name='Chen', value_name='Amp')
Hem = {'P8_amp_ob' : 'R','P7_amp_ob': 'L', 'O1_amp_ob': 'L','O2_amp_ob': 'R',
      'P8_amp_bas': 'R', 'P7_amp_bas' : 'L','O1_amp_bas': 'L', 'O2_amp_bas' : 'R'}

Reg= {'P8_amp_ob' : 'P', 'P7_amp_ob': 'P', 'O1_amp_ob': 'O', 'O2_amp_ob': 'O',
      'P8_amp_bas': 'P', 'P7_amp_bas' : 'P', 'O1_amp_bas': 'O', 'O2_amp_bas' : 'O'}
Fr= {'P8_amp_ob' : 'ob', 'P7_amp_ob': 'ob','O1_amp_ob': 'ob', 'O2_amp_ob': 'ob',
      'P8_amp_bas': 'bas', 'P7_amp_bas' : 'bas', 'O1_amp_bas': 'bas', 'O2_amp_bas' : 'bas'}
df['Hem'] = df['Chen'].map(Hem) 
df['Reg'] = df['Chen'].map(Reg) 
df['Fr'] = df['Chen'].map(Fr) 
df['Cond'] = "for"
df_for = df[['ID','Cond','Hem','Reg','Fr','Amp']]
df_for

In [ ]:
psds_list=load('inv_psds.npy')
freqs_list=load('inv_freqs.npy')
snrs_list=load('inv_snrs.npy')
zscrs_list=load('inv_zscrs.npy')
amp_cor_list=load('inv_amp_cor.npy')

wal = dict()
freq_plot = [[1.2, 2.4, 3.6],[6, 12, 18]]
ch_P8=['P8']
ch_P7=['P7']
ch_O1=['O1']
ch_O2=['O2']

for j in range (0, len(EEG)):    
    res = dict()
    picks_P8 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P8)
    picks_P7 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_P7)
    picks_O1 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O1)
    picks_O2 = mne.pick_types(EEG[i].info, eeg=True, stim=False,
                               exclude='bads', selection=ch_O2)

    zscrs_roi_P8 = amp_cor_list[j][ picks_P8, :]
    zscrs_roi_P7 = amp_cor_list[j][ picks_P7, :]
    zscrs_roi_O1 = amp_cor_list[j][ picks_O1, :]
    zscrs_roi_O2 = amp_cor_list[j][ picks_O2, :]
    
    P8_list=[]
    P7_list=[]
    O1_list=[]
    O2_list=[]
    
    for i, f in enumerate(freq_plot[0]):
    # extract snrs
        stim_P8_tmp = \
            zscrs_roi_P8[:, np.argmin(abs(av_freqs - f))][0]
        P8_list.append(stim_P8_tmp)
        stim_P7_tmp = \
            zscrs_roi_P7[:, np.argmin(abs(av_freqs - f))][0]
        P7_list.append(stim_P7_tmp)
        stim_O1_tmp = \
            zscrs_roi_O1[:, np.argmin(abs(av_freqs - f))][0]
        O1_list.append(stim_O1_tmp)
        stim_O2_tmp = \
            zscrs_roi_O2[:, np.argmin(abs(av_freqs - f))][0]
        O2_list.append(stim_O2_tmp)

    res['P8_amp_ob'] = sum(P8_list)
    res['P7_amp_ob']  = sum(P7_list)
    res['O1_amp_ob'] = sum(O1_list)
    res['O2_amp_ob'] = sum(O2_list)
    P8_list=[]
    P7_list=[]
    O1_list=[]
    O2_list=[]
    for i, f in enumerate(freq_plot[1]):
    # extract snrs
        stim_P8_tmp = \
            zscrs_roi_P8[:, np.argmin(abs(av_freqs - f))][0]
        P8_list.append(stim_P8_tmp)
        stim_P7_tmp = \
            zscrs_roi_P7[:, np.argmin(abs(av_freqs - f))][0]
        P7_list.append(stim_P7_tmp)
        stim_O1_tmp = \
            zscrs_roi_O1[:, np.argmin(abs(av_freqs - f))][0]
        O1_list.append(stim_O1_tmp)
        stim_O2_tmp = \
            zscrs_roi_O2[:, np.argmin(abs(av_freqs - f))][0]
        O2_list.append(stim_O2_tmp)

    res['P8_amp_bas'] = sum(P8_list)
    res['P7_amp_bas']  = sum(P7_list)
    res['O1_amp_bas'] = sum(O1_list)
    res['O2_amp_bas'] = sum(O2_list)
    wal[ID[j]]=res

df=pd.DataFrame.from_dict(wal, orient='index')
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'ID'})
df = pd.melt(df, id_vars=['ID'], value_vars=['P8_amp_ob', 'P7_amp_ob', 'O1_amp_ob', 'O2_amp_ob', 
                                             'P8_amp_bas', 'P7_amp_bas', 'O1_amp_bas', 'O2_amp_bas'],
        var_name='Chen', value_name='Amp')
dic = {'P8_amp_ob' : 'R','P7_amp_ob': 'L', 'O1_amp_ob': 'L','O2_amp_ob': 'R',
      'P8_amp_bas': 'R', 'P7_amp_bas' : 'L','O1_amp_bas': 'L', 'O2_amp_bas' : 'R'}

Reg= {'P8_amp_ob' : 'P', 'P7_amp_ob': 'P', 'O1_amp_ob': 'O', 'O2_amp_ob': 'O',
      'P8_amp_bas': 'P', 'P7_amp_bas' : 'P', 'O1_amp_bas': 'O', 'O2_amp_bas' : 'O'}
Fr= {'P8_amp_ob' : 'ob', 'P7_amp_ob': 'ob','O1_amp_ob': 'ob', 'O2_amp_ob': 'ob',
      'P8_amp_bas': 'bas', 'P7_amp_bas' : 'bas', 'O1_amp_bas': 'bas', 'O2_amp_bas' : 'bas'}
df['Hem'] = df['Chen'].map(Hem) 
df['Reg'] = df['Chen'].map(Reg) 
df['Fr'] = df['Chen'].map(Fr) 
df['Cond'] = "inv"
df_inv = df[['ID','Cond','Hem','Reg','Fr','Amp']]
df_inv

In [ ]:
df= pd.concat([df_for, df_inv])
df.reset_index(drop= True , inplace= True )
df

In [ ]:
df_inv

In [ ]:
grouped = df.groupby(['Cond','Fr','ID'], as_index=False)
mean=grouped.mean()
type(mean)

In [ ]:
freqs_list=load('for_freqs.npy')
for_psds_list=load('for_psds.npy')
for_snrs_list=load('for_snrs.npy')
for_zscrs_list=load('for_zscrs.npy')
for_amp_cor_list=load('for_amp_cor.npy')
inv_psds_list=load('inv_psds.npy')
inv_snrs_list=load('inv_snrs.npy')
inv_zscrs_list=load('inv_zscrs.npy')
inv_amp_cor_list=load('inv_amp_cor.npy')

In [ ]:
ind_ok=[ID.index(i) for i in ID_ok]
for_psds_ok=for_psds_list[ind_ok]
for_snrs_ok= for_snrs_list[ind_ok] 
for_zscrs_ok= for_zscrs_list[ind_ok]
for_amp_cor_ok= for_amp_cor_list[ind_ok]
inv_psds_ok= inv_psds_list[ind_ok]
inv_snrs_ok= inv_snrs_list[ind_ok]
inv_zscrs_ok= inv_zscrs_list[ind_ok]
inv_amp_cor_ok= inv_amp_cor_list[ind_ok]

In [ ]:
i_bin_1_2hz = np.argmin(abs(av_freqs - 1.2))
i_bin_2_4hz = np.argmin(abs(av_freqs - 2.4))
i_bin_3_6hz = np.argmin(abs(av_freqs - 3.6))
i_bin_4_8hz = np.argmin(abs(av_freqs - 4.8))

i_bin_6hz = np.argmin(abs(av_freqs - 6))
i_bin_12hz = np.argmin(abs(av_freqs - 12))
i_bin_18hz = np.argmin(abs(av_freqs - 18))
i_bin_24hz = np.argmin(abs(av_freqs - 24))

snrs_1_2hz = av_zscrs[:, i_bin_1_2hz]
snrs_1_2hz_chaverage = snrs_1_2hz
snrs_6hz = av_zscrs[:, i_bin_6hz]
snrs_6hz_chaverage = snrs_6hz

snrs_2_4hz = av_zscrs[:, i_bin_2_4hz]
snrs_2_4hz_chaverage = snrs_2_4hz
snrs_12hz = av_zscrs[:, i_bin_12hz]
snrs_12hz_chaverage = snrs_12hz

snrs_3_6hz = av_zscrs[:, i_bin_3_6hz]
snrs_3_6hz_chaverage = snrs_3_6hz
snrs_18hz = av_zscrs[:, i_bin_18hz]
snrs_18hz_chaverage = snrs_18hz


mask_1_2hz=np.copy(snrs_1_2hz_chaverage)
np.place(mask_1_2hz, mask_1_2hz<1.64, [0])
np.place(mask_1_2hz, mask_1_2hz>1.64, [1])
mask_2_4hz=np.copy(snrs_2_4hz_chaverage)
np.place(mask_2_4hz, mask_2_4hz<1.64, [0])
np.place(mask_2_4hz, mask_2_4hz>1.64, [1])
mask_3_6hz=np.copy(snrs_3_6hz_chaverage)
np.place(mask_3_6hz, mask_3_6hz<1.64, [0])
np.place(mask_3_6hz, mask_3_6hz>1.64, [1])
mask_6hz=np.copy(snrs_6hz_chaverage)
np.place(mask_6hz, mask_6hz<1.64, [0])
np.place(mask_6hz, mask_6hz>1.64, [1])
mask_12hz=np.copy(snrs_12hz_chaverage)
np.place(mask_12hz, mask_12hz<1.64, [0])
np.place(mask_12hz, mask_12hz>1.64, [1])
mask_18hz=np.copy(snrs_18hz_chaverage)
np.place(mask_18hz, mask_18hz<1.64, [0])
np.place(mask_18hz, mask_18hz>1.64, [1])

mask_p=dict(markersize=15, markerfacecolor="w")
      

fig, ax = plt.subplots(2,3, figsize=[12, 9])
mne.viz.plot_topomap(snrs_1_2hz_chaverage, EEG[0].info, names=EEG[0].ch_names, 
                     mask=mask_1_2hz, mask_params=mask_p,
                     cmap='RdPu', ch_type="eeg", sensors=True, vlim=(1, 8), axes=ax[0,0])
mne.viz.plot_topomap(snrs_2_4hz_chaverage, EEG[0].info, names=EEG[0].ch_names, 
                     mask=mask_2_4hz, mask_params=mask_p,
                     cmap='RdPu', ch_type="eeg", sensors=True, vlim=(1, 8), axes=ax[0,1])
mne.viz.plot_topomap(snrs_3_6hz_chaverage, EEG[0].info, names=EEG[0].ch_names, 
                     mask=mask_3_6hz, mask_params=mask_p,
                     cmap='RdPu', ch_type="eeg", sensors=True, vlim=(1, 8), axes=ax[0,2])


mne.viz.plot_topomap(snrs_6hz_chaverage, EEG[0].info, names=EEG[0].ch_names, 
                     mask=mask_6hz, mask_params=mask_p,
                     cmap='RdPu', ch_type="eeg", sensors=True, vlim=(1, 8), axes=ax[1,0])
mne.viz.plot_topomap(snrs_12hz_chaverage, EEG[0].info, names=EEG[0].ch_names, 
                     mask=mask_12hz, mask_params=mask_p,
                     cmap='RdPu', ch_type="eeg", sensors=True, vlim=(1, 8),axes=ax[1,1])
mne.viz.plot_topomap(snrs_18hz_chaverage, EEG[0].info, names=EEG[0].ch_names, 
                     mask=mask_18hz, mask_params=mask_p,
                     cmap='RdPu', ch_type="eeg", sensors=True, vlim=(1, 8),axes=ax[1,2])


ax[0,0].set_title('1.2 Hz Z scores')
ax[0,1].set_title('2.4 Hz Z scores')
ax[0,2].set_title('3.6 Hz Z scores')

ax[1,0].set_title('6 Hz Z scores')
ax[1,1].set_title('12 Hz Z scores')
ax[1,2].set_title('18 Hz Z scores')


fig.suptitle(f'Foreign', fontsize=16)#В ковычках можем прописать заголовок
fig.show()
#fig.savefig(f'Z_Topo_Invers.png')

In [ ]:
snrs_3_6hz_chaverage

In [ ]:
snrs_1_2hz_chaverage

In [ ]:
list1=[]
list2=[]
list6=[]
list12=[]
for a in mask_1_2hz:
    list1.append(EEG[0].ch_names[a])
for a in mask_2_4hz:
    list2.append(EEG[0].ch_names[a])
for a in mask_6hz:
    list6.append(EEG[0].ch_names[a])
for a in mask_12hz:
    list12.append(EEG[0].ch_names[a])
print(list1)
print(list2)
print(list6)
print(list12)    

In [ ]:
from matplotlib import style
plt.style.use("seaborn-colorblind")

In [ ]:
all_bloks=[eplist_1, eplist_2, eplist_3, eplist_4, eplist_5, eplist_6]
for n in range (1,7):

    target_list=all_bloks[n-1]
    tmin = 1. 
    tmax = 120.
    fmin = 1.
    fmax = 30.
    sfreq = 500
    psds_list = []
    freqs_list = []
    snrs_list = []

    for i in range (0, len(EEG)):
        spectrum = target_list[i].compute_psd(
        'welch',
        n_fft=int(sfreq * (tmax - tmin)),
        n_overlap=0, n_per_seg=None,
        tmin=tmin, tmax=tmax,
        fmin=fmin, fmax=fmax,
        window='boxcar',
        verbose=False)
        psds, freqs = spectrum.get_data(return_freqs=True)

        psds_list.append(psds)
        freqs_list.append(freqs)

        snrs = snr_spectrum(psds, noise_n_neighbor_freqs=10,
                        noise_skip_neighbor_freqs=1)
        snrs_list.append(snrs)  
    av_psds = np.mean(psds_list, axis=0)
    av_freqs = np.mean(freqs_list, axis=0)
    av_snrs = np.mean(snrs_list, axis=0)    
    i_bin_12hz = np.argmin(abs(av_freqs - 1.2))
    i_bin_6hz = np.argmin(abs(av_freqs - 6))
    i_bin_8hz = np.argmin(abs(av_freqs - 8))
    i_trial_12hz = np.where(eplist[0].events[:, 2] == 1)[0]

#Рисуем усредненную картинку для всех людей
    fig, axes = plt.subplots(2, 1, sharex='all', sharey='none', figsize=(18, 10))
    freq_range = range(np.where(np.floor(av_freqs) == 1.)[0][0],
                   np.where(np.ceil(av_freqs) == fmax - 1)[0][0])

    psds_plot = 10 * np.log10(av_psds)
    psds_mean = psds_plot.mean(axis=(0, 1))[freq_range]
    psds_std = psds_plot.std(axis=(0, 1))[freq_range]
    axes[0].plot(av_freqs[freq_range], psds_mean, color='C2')
    axes[0].fill_between(
    av_freqs[freq_range], psds_mean - psds_std, psds_mean + psds_std,
    color='C2', alpha=.2)
    axes[0].set(title="PSD spectrum", ylabel='Power Spectral Density [dB]')

# SNR spectrum
    snr_mean = av_snrs.mean(axis=(0, 1))[freq_range]
    snr_std = av_snrs.std(axis=(0, 1))[freq_range]

    axes[1].plot(av_freqs[freq_range], snr_mean, color='C1')
    axes[1].fill_between(
        av_freqs[freq_range], snr_mean - snr_std, snr_mean + snr_std,
        color='C1', alpha=.2)
    axes[1].set(
        title="SNR spectrum", xlabel='Frequency [Hz]',
        ylabel='SNR', ylim=[0, 15], xlim=[fmin, 25])

    axes[0].axvspan(1.2-0.02, 1.2+0.02, alpha=0.2, color ='k')
    axes[0].axvspan(1.2*2-0.02, 1.2*2+0.02, alpha=0.2, color ='k')
    axes[0].axvspan(1.2*3-0.02, 1.2*3+0.02, alpha=0.2, color ='k')
    axes[0].axvspan(1.2*4-0.02, 1.2*4+0.02, alpha=0.2, color ='k')
    axes[0].axvspan(6-0.02, 6+0.02, alpha=0.2, color ='b')
    axes[0].axvspan(6*2-0.02, 6*2+0.02, alpha=0.2, color ='b')
    axes[0].axvspan(6*3-0.02, 6*3+0.02, alpha=0.2, color ='b')
    axes[0].axvspan(6*4-0.02, 6*4+0.02, alpha=0.2, color ='b')

    axes[1].axvspan(1.2-0.02, 1.2+0.02, ymin=0, ymax=5, alpha=0.2, color ='k')
    axes[1].axvspan(1.2*2-0.02, 1.2*2+0.02, ymin=0, ymax=5, alpha=0.2, color ='k')
    axes[1].axvspan(1.2*3-0.02, 1.2*3+0.02, ymin=0, ymax=5, alpha=0.2, color ='k')
    axes[1].axvspan(1.2*4-0.02, 1.2*4+0.02, ymin=0, ymax=5, alpha=0.2, color ='k')
    axes[1].axvspan(6-0.02, 6+0.02, ymin=0, ymax=5, alpha=0.2, color ='b')
    axes[1].axvspan(6*2-0.02, 6*2+0.02, ymin=0, ymax=5, alpha=0.2, color ='b')
    axes[1].axvspan(6*3-0.02, 6*3+0.02, ymin=0, ymax=5, alpha=0.2, color ='b')
    axes[1].axvspan(6*4-0.02, 6*4+0.02, ymin=0, ymax=5, alpha=0.2, color ='b')


    fig.suptitle(f'Block {str(n)}', fontsize=16)#В ковычках можем прописать заголовок
    fig.show()
    fig.savefig(f'PSD_{str(n)}_25.png')

In [ ]:
all_bloks=[eplist_1, eplist_2, eplist_3, eplist_4, eplist_5, eplist_6]
for n in range (1,7):

    target_list=all_bloks[n-1]
    tmin = 1. 
    tmax = 120.
    fmin = 1.
    fmax = 30.
    sfreq = 500
    psds_list = []
    freqs_list = []
    snrs_list = []

    for i in range (0, len(EEG)):
        spectrum = target_list[i].compute_psd(
        'welch',
        n_fft=int(sfreq * (tmax - tmin)),
        n_overlap=0, n_per_seg=None,
        tmin=tmin, tmax=tmax,
        fmin=fmin, fmax=fmax,
        window='boxcar',
        verbose=False)
        psds, freqs = spectrum.get_data(return_freqs=True)

        psds_list.append(psds)
        freqs_list.append(freqs)

        snrs = snr_spectrum(psds, noise_n_neighbor_freqs=10,
                    noise_skip_neighbor_freqs=1)
        snrs_list.append(snrs)  
    av_psds = np.mean(psds_list, axis=0)
    av_freqs = np.mean(freqs_list, axis=0)
    av_snrs = np.mean(snrs_list, axis=0)    
    #freq_plot = [ 3.9, 5.3]
#color_plot = ['C5', 'C6']
    #xpos_plot = [-0.2, 0.2]
    freq_plot = [1.2, 2.4, 3.6, 4.8, 6, 12, 18, 24]
#color_plot = ['C5', 'C6']
    xpos_plot = [ -0.8,-0.6,-0.4,-0.2, 0.2,0.4,0.6,0.8]
    fig, ax = plt.subplots(figsize=[10, 8])
    labels = ['Parietal Right', 'Parietal Left', 'Frontal Right', 'Frontal Left' ]
    x = np.arange(len(labels))  # the label locations
    width = 0.3 # the width of the bars
    res = dict()

# loop to plot SNRs at stimulation frequencies and harmonics
    for i, f in enumerate(freq_plot):
    # extract snrs
        stim_pright_tmp = \
            snrs_roi_pr[i_trial_12hz, np.argmin(abs(freqs - f))]
        stim_pleft_tmp = \
            snrs_roi_pl[i_trial_12hz, np.argmin(abs(freqs - f))]
        stim_fright_tmp = \
            snrs_roi_pr[i_trial_12hz, np.argmin(abs(freqs - f))]
        stim_fleft_tmp = \
            snrs_roi_pl[i_trial_12hz, np.argmin(abs(freqs - f))]
    
        SNR_tmp = [stim_pright_tmp.mean(), stim_pleft_tmp.mean(), stim_fright_tmp.mean(), stim_fleft_tmp.mean()]
    # plot (with std)
        ax.bar(
            x + width * xpos_plot[i], SNR_tmp, width / len(freq_plot),
            yerr=np.std(SNR_tmp),
            label='%i Hz SNR' % f)
    # store results for statistical comparison
        res['stim_pright_snrs_%ihz' % f] = stim_pright_tmp[0]
        res['stim_pleft_snrs_%ihz' % f] = stim_pleft_tmp[0]
        res['stim_fright_snrs_%ihz' % f] = stim_fright_tmp[0]
        res['stim_fleft_snrs_%ihz' % f] = stim_fleft_tmp[0]


# Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('SNR')
    ax.set_title(f'Average SNR \n Block {str(n)}')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend([f'{f} Hz' for f in freq_plot], title='SNR at:')
    ax.set_ylim([0, 2.5])
    ax.axhline(1, ls='--', c='r')
    fig.show()
    #fig.savefig(f'SNR_{str(n)}_main.png')

In [ ]:
for Id in ID:
    df=pd.read_csv(f'VFT_logs/{Id}_VFT.csv', header=0)
    if Id == 'NP102309':
        df=df[df["trials.thisRepN"] == 0][3:723]
    else: df=df[df["trials_2.thisRepN"] == 0][3:723]    
    x=df['tr'].to_numpy()
    z=df['key_resp_2.keys'].to_numpy()
    z[z == 'space'] = 1 
    z[z == 'None'] = 0 
    targets= [129, 485, 217, 114, 711, 23, 325, 635, 69, 47, 705, 145, 600, 384, 130, 128, 486, 484, 218, 216, 115, 113, 712, 710, 24, 22, 326, 324, 636, 634, 70, 68, 48, 46, 706, 704, 146, 144, 601, 599, 385, 383]
    tr_ar=[]
    for i in x:
        if i in targets:
            tr_ar.append(2)
        else: tr_ar.append(0)  
    y=np.array(tr_ar)
    fig, ax = plt.subplots(figsize=[20, 2])
    plt.fill_between(x, 0, y,color='darkblue', alpha=0.5, axes=ax)
    plt.scatter(x,z,color='red', axes=ax)
    ax.set(ylim=[0.9, 1.1])
    fig.suptitle(f'{Id}', fontsize=16)
    fig.savefig(f'{Id}_for_resp.png')
#plt.plot(x,z,color='darkorange', linewidth=4, label='1.2Hz')

In [ ]:
for Id in ID:
    df=pd.read_csv(f'VFT_logs/{Id}_VFT.csv', header=0)
    if Id == 'NP102309':
        df=df[df["trials.thisRepN"] == 0][3:723]
    else: df=df[df["trials_2.thisRepN"] == 0][3:723]    
    x=df['tr'].to_numpy()
    z=df['key_resp_4.keys'].to_numpy()
    z[z == 'space'] = 1 
    z[z == 'None'] = 0 
    targets= [589, 89, 401, 142, 123, 85, 117, 692, 396, 706, 663, 317, 549, 702, 590, 588, 90, 88, 402, 400, 143, 141, 124, 122, 86, 84, 118, 116, 693, 691, 397, 395, 707, 705, 664, 662, 318, 316, 550, 548, 703, 701]
    tr_ar=[]
    for i in x:
        if i in targets:
            tr_ar.append(2)
        else: tr_ar.append(0)  
    y=np.array(tr_ar)
    fig, ax = plt.subplots(figsize=[20, 2])
    plt.fill_between(x, 0, y,color='gold', alpha=0.5, axes=ax)
    plt.scatter(x,z,color='red', axes=ax)
    ax.set(ylim=[0.9, 1.1])
    fig.suptitle(f'{Id}', fontsize=16)
    fig.savefig(f'{Id}_inv_resp.png')

In [ ]:
df=pd.read_csv(f'VFT_logs/{ID[5]}_VFT.csv', header=0)
df=df[df["trials_g.thisRepN"] == 0][3:723]
x=df['tr'].to_numpy()
z=df['key_resp_2.keys'].to_numpy()
z[z == 'space'] = 1 
z[z == 'None'] = 0 
targets = [129, 485, 217, 114, 711, 23, 325, 635, 69, 47, 705, 145, 600, 384]
targets.sort()
d=[]
for t in targets:
    for i in range (t, len(z)):
        if z[i]==1: 
            break
    d.append(i-t) 
d    

In [ ]:
res = dict()
raw_ind=0
for Id in ID:
    df=pd.read_csv(f'VFT_logs/{Id}_VFT.csv', header=0)
    df=df[df["trials_g.thisRepN"] == 0][3:723]
    z=df['key_resp_2.keys'].to_numpy()
    z[z == 'space'] = 1 
    z[z == 'None'] = 0 
    targets = [129, 485, 217, 114, 711, 23, 325, 635, 69, 47, 705, 145, 600, 384]
    targets.sort()
    d=[]
    war = dict()
    for j in range(len(targets)):
        war = dict()
        t=targets[j]
        for i in range (t, len(z)):
            if z[i]==1: 
                break
        d.append(i-t)
        war['Id']=Id
        war['star']=j+1
        war['del']=i-t
        raw_ind=raw_ind+1
        res[raw_ind]=war
   # y=np.array(d) 
   # x = [i for i in range(len(targets))]
    #fig, ax = plt.subplots()
    #plt.scatter(x,y,color='red', axes=ax)
    #fig.suptitle(f'{Id}', fontsize=16)
    #fig.savefig(f'{Id}_for_del.png')
df=pd.DataFrame.from_dict(res,orient='index')
df
sns.barplot(x=df['star'], y=df['del'], data=df)

In [ ]:
res = dict()
raw_ind=0
for Id in ID:
    df=pd.read_csv(f'VFT_2_new_logs/{Id}_VFT.csv', header=0)
    if Id == 'NP102309':
        df=df[df["trials.thisRepN"] == 0][3:723]
    else: df=df[df["trials_2.thisRepN"] == 0][3:723]    
    x1=df['tr'].to_numpy()
    z=df['key_resp_4.keys'].to_numpy()
    z[z == 'space'] = 1 
    z[z == 'None'] = 0 
    targets = [588, 88, 400, 141, 122, 84, 116, 691, 395, 705, 662, 316, 548, 701]
    targets.sort()
    d=[]
    x=df['tr'].to_numpy()
    targets_all= [589, 89, 401, 142, 123, 85, 117, 692, 396, 706, 663, 317, 549, 702, 590, 588, 90, 88, 402, 400, 143, 141, 124, 122, 86, 84, 118, 116, 693, 691, 397, 395, 707, 705, 664, 662, 318, 316, 550, 548, 703, 701]
    tr_ar=[]
    for i in x1:
        if i in targets_all:
            tr_ar.append(2)
        else: tr_ar.append(0)  
    y1=np.array(tr_ar)

    for j in range(len(targets)):
        war = dict()
        t=targets[j]
        for i in range (t, len(z)):
            if z[i]==1: 
                break
        d.append(i-t)
        war['Id']=Id
        war['star']=j+1
        war['del']=i-t
        raw_ind=raw_ind+1
        res[raw_ind]=war
    y2=np.array(d) 
    x2 = [i for i in range(len(targets))]
    fig, ax = plt.subplots(figsize=[20, 5])
    ax1 = fig.add_subplot(2, 2, (1,2))
    ax2 = fig.add_subplot(223)
    ax1.fill_between(x1, 0, y1,color='darkblue', alpha=0.5)
    ax1.scatter(x1,z,color='red')
    ax2.scatter(x2,y2,color='red')
    ax1.set(ylim=[0.8, 1.2])
    for i in range(len(targets)):
        ax1.text(targets[i]-2, 0.9, str(i+1), fontsize=12)
    ax2.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13])
    ax2.set_xticklabels(['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])
    ax.axis('off')
    ax2.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13])
    ax2.set_xticklabels(['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])
    fig.suptitle(f'{Id}', fontsize=16)
    fig.savefig(f'{Id}_inv_m.png')
df=pd.DataFrame.from_dict(res,orient='index')
df
#

In [ ]:
res = dict()
raw_ind=0
for Id in ID:
    df=pd.read_csv(f'VFT_2_new_logs/{Id}_VFT.csv', header=0)
    df=df[df["trials_g.thisRepN"] == 0][3:723]
    z=df['key_resp_2.keys'].to_numpy()
    z[z == 'space'] = 1 
    z[z == 'None'] = 0 
    targets = [128, 484, 216, 113, 710, 22, 324, 634, 68, 46, 704, 144, 599, 383]
    targets.sort()
    d=[]
    x1=df['tr'].to_numpy()
    targets_all=  [129, 485, 217, 114, 711, 23, 325, 635, 69, 47, 705, 145, 600, 384, 130, 128, 486, 484, 218, 216, 115, 113, 712, 710, 24, 22, 326, 324, 636, 634, 70, 68, 48, 46, 706, 704, 146, 144, 601, 599, 385, 383]
    tr_ar=[]
    for i in x1:
        if i in targets_all:
            tr_ar.append(2)
        else: tr_ar.append(0)  
    y1=np.array(tr_ar)

    for j in range(len(targets)):
        war = dict()
        t=targets[j]
        for i in range (t, len(z)):
            if z[i]==1: 
                break
        d.append(i-t)
        war['Id']=Id
        war['star']=j+1
        war['del']=i-t
        raw_ind=raw_ind+1
        res[raw_ind]=war
    y2=np.array(d) 
    x2 = [i for i in range(len(targets))]
    fig, ax = plt.subplots(figsize=[20, 5])
    ax1 = fig.add_subplot(2, 2, (1,2))
    ax2 = fig.add_subplot(223)
    ax1.fill_between(x1, 0, y1,color='darkblue', alpha=0.5)
    ax1.scatter(x1,z,color='red')
    ax2.scatter(x2,y2,color='red')
    ax1.set(ylim=[0.8, 1.2])
    for i in range(len(targets)):
        ax1.text(targets[i]-2, 0.9, str(i+1), fontsize=12)
    ax.axis('off')
    ax2.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13])
    ax2.set_xticklabels(['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])
    fig.suptitle(f'{Id}', fontsize=16)
    fig.savefig(f'{Id}_for_m.png')
df=pd.DataFrame.from_dict(res,orient='index')
df
#sns.boxplot(x=df['star'], y=df['del'], data=df)

In [ ]:
df.describe()

In [ ]:
df_id=df.pivot(index='star', columns='Id', values='Hits')
df_id=df_id.transpose()
df_id["Hits"]=df_id.sum(axis=1)
df_id
df_id.to_excel('Inv_hits_new.xlsx', sheet_name='Sheet1')